In [60]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import re
from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [23]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [57]:
def plot_result(item):
    plt.plot(history.history[item], label=item)
    plt.plot(history.history["val_" + item], label="val_" + item)
    plt.xlabel("Epochs")
    plt.ylabel(item)
    plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
    plt.legend()
    plt.grid()
    plt.show()


plot_result("loss")
plot_result("categorical_accuracy")

NameError: name 'history' is not defined

In [32]:
df = pd.read_csv("data/books_def.csv", index_col=0)

In [61]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [64]:
X = []
sentences = list(df["book_desc"])
for sen in sentences:
    X.append(preprocess_text(sen))

y = df.genres.values

36638

In [33]:
# leggo testi & etichette
test_split = 0.25

# Initial train and test split.
train_df, test_df = train_test_split(
    df,
    test_size=test_split,
    stratify=df["genres"].values,
)

# Splitting the test set further into validation
# and new test sets.
val_df = test_df.sample(frac=0.5)
test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 27478
Number of rows in validation set: 4580
Number of rows in test set: 3109


<ipython-input-33-ba8aa7f94dae>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(val_df.index, inplace=True)


In [35]:
terms = tf.ragged.constant(train_df["genres"].values)
lookup = tf.keras.layers.StringLookup(output_mode="multi_hot")
lookup.adapt(terms)
vocab = lookup.get_vocabulary()


def invert_multi_hot(encoded_labels):
    """Reverse a single multi-hot encoded label to a tuple of vocab terms."""
    hot_indices = np.argwhere(encoded_labels == 1.0)[..., 0]
    return np.take(vocab, hot_indices)


print("Vocabulary:\n")
print(vocab)

Vocabulary:

['[UNK]', 'Fiction', 'Nonfiction', 'Fantasy', 'Romance', 'Young Adult', 'Historical', 'Mystery', 'Science Fiction', 'Classics', 'Sequential Art']


In [36]:
sample_label = train_df["genres"].iloc[0]
print(f"Original label: {sample_label}")

label_binarized = lookup([sample_label])
print(f"Label-binarized representation: {label_binarized}")

Original label: Young Adult
Label-binarized representation: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [39]:
# we check the 2nd percentile: any number close to that value is a good enough approximate for the maximum sequence length.

In [38]:
train_df["book_desc"].apply(lambda x: len(x.split(" "))).describe()


count    27478.000000
mean       143.482459
std         84.679790
min          1.000000
25%         89.000000
50%        133.000000
75%        181.000000
max       2861.000000
Name: book_desc, dtype: float64

In [55]:
# Instanciation du Tokenizer avec les paramètres
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
# Adaptation du Tokenizer au jeu d'entrainement
tokenizer.fit_on_texts(X_train)
# Création d'un dictionnaire d'index des mots
word_index = tokenizer.word_index                             

# Conversion en sequences d'entier
X_train_sequences = tokenizer.texts_to_sequences(X_train)

# Ajout du padding 
# Objectif: permettre à chaque séquence de possèder la même longueur
X_train_padded = pad_sequences(X_train_sequences, maxlen = max_length, truncating = trunc_type)

In [56]:
train_dataset = make_dataset(train_df, is_train=True)
validation_dataset = make_dataset(val_df, is_train=False)
test_dataset = make_dataset(test_df, is_train=False)

ValueError: Dimensions 36638 and 11 are not compatible